In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import pickle
import datetime
from IPython.display import HTML, display
import tabulate
from copy import deepcopy
from gene import Gene, GeneChain
from evolution import Evolution
from loader import load_results, load_trading_params, load_nontrading_params, load_indivs
from crosser import Cross10to100, RandomGeneCrosser
from mutator import GausianMutator, MutRandomIndivs, RandomGeneChainMutator
from selector import TopNSelector
from util import plot
from fitness import Fitness

## Load data

In [3]:
db_indivs = load_indivs()   # pandas object, for fitness function; db individuals have only non-trading parameters
trading_params = load_trading_params()  # pandas object, for precomputation
nontrading_params = load_nontrading_params()    # pandas object, for precomputation

## Init base gene chain

In [4]:
base_gene_chain = GeneChain()
gene_descriptors = []
for index, row in nontrading_params.iterrows():
    dtype = np.dtype(row["TYPE"]).type
    is_interval = True if dtype != bool else False
    min_range = dtype(row["MIN_RANGE"])
    max_value = dtype(max(db_indivs[index]))
    min_value = dtype(min(db_indivs[index]))        
    gene_descriptors.append(str(row["DESCRIPTION"]))
    base_gene_chain.add(Gene(dtype=dtype, is_interval=is_interval, min_radius=min_range/2,
                             min_value=min_value, max_value=max_value, ordering=-1))    

for index, row in trading_params.iterrows():
    is_interval = False
    dtype = np.dtype(row["TYPE"]).type
    min_value = dtype(row["MIN"])
    max_value = dtype(row["MAX"])
    gene_descriptors.append(str(row["DESCRIPTION"]))
    base_gene_chain.add(Gene(dtype=dtype, is_interval=is_interval, tags=["trading"],
                             min_value=min_value, max_value=max_value, ordering=-1))    

## Init genetic algorithm

In [5]:
ev = Evolution(
    MAX_GENS=30,
    fitness=Fitness(),
    base_indiv=base_gene_chain,
    crosser=RandomGeneCrosser(),
    selector=TopNSelector(),
    verbose=True)

## Create start population

In [23]:
ev.init_random_indivs(size=10, min_models=100)
ev.dump("indivs.pkl")
#ev.load_indivs("indivs.pkl")

Found 10 indivs with 2000 tries (3s.)                                                                                                    

## Run 

In [ ]:
ev.run(dump="best.pkl")

In [ ]:
plot([ res["fitness"].fitness for res in ev._run_results ], "fitness")

In [ ]:
ev.print_generation()

## Working with results

In [ ]:
fitness = FitnessTest()

In [ ]:
with open("indivs/best1.pkl", 'rb') as file:
    indivs = pickle.load(file)
indiv = indivs[0]
indiv.fitness = None
fitness.calc([indiv], ext_info=True)

print(indiv.fitness)
plot(indiv.fitness.fitness_list)

In [ ]:
compare_trading_reports(fitness, indiv.fitness)

In [ ]:
table = [["#", "Line", "Time", "Type", "Order", "Price", "S/L", "T/P"]]
table.extend(indiv.fitness.trade_history)
display(HTML(tabulate.tabulate(table[:100], tablefmt='html')))

In [ ]:
print_descr = False
count_empty_models = False
print("Indiv parameters:")
for gene_idx, gene in enumerate(indiv):
    if print_descr:
        print(gene_descriptors[gene_idx] + ":\n" + str(gene))
    else:
        print(gene)          
        
print("Inidiv models:")
fit_params = indiv.fitness
trades = 0
table = [["model number", "stops", "take", "bu"]]
for idx in range(len(fit_params.model_indexes)):
    if not count_empty_models and fit_params.stops[idx] == 0 and \
       fit_params.takes[idx] == 0 and fit_params.bus[idx] == 0:
        continue
    trades += fit_params.stops[idx]
    trades += fit_params.takes[idx]
    trades += fit_params.bus[idx]
    table.append([fit_params.model_indexes[idx], fit_params.stops[idx], fit_params.takes[idx], fit_params.bus[idx]])
print("total models number: " + str(len(table)))
print("total trades number: " + str(trades))
#display(HTML(tabulate.tabulate(table, tablefmt='html')))